In [1]:
# 1) Business Question, # 2) Data Collection, # 3) Preprocessing, # 4) Model(s) Creation, # 5) Model Evaluation and Comparison , # 6) Conclusion and Future Improvements
#https://www.kaggle.com/c/home-credit-default-risk/data?select=installments_payments.csv

In [2]:
#Dataframe
import pandas as pd
import numpy as np
import math
from datetime import datetime
import scipy
from scipy import stats

# To Plot Pretty Figures:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

plt.rcParams['figure.figsize'] = (13, 8)    # set default figure size...must set after matplotlib inline on a DIFFERENT cell
plt.style.use('seaborn-deep')       # change color palette

#sklearn & models
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import time

#Data processing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from  sklearn.impute import SimpleImputer   
from imblearn.over_sampling import SMOTE

#   Print 30 lines
pd.set_option('display.max_rows', 30)
#   allowing for any single variable to print out without using the print statement:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# to make this notebook's output stable across runs
np.random.seed(42)


In [50]:
df_train = pd.read_csv('./data/application_train.csv')
df_bureau = pd.read_csv('./data/bureau.csv')
df_bureau_bal = pd.read_csv('./data/bureau_balance.csv')
df_CC = pd.read_csv('./data/credit_card_balance.csv')
df_POS = pd.read_csv('./data/POS_CASH_balance.csv')
df_pre = pd.read_csv('./data/previous_application.csv')

In [4]:
df_train

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df_bureau

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,11250.00,11250.0,0.0,0.0,Microloan,-19,NaN
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.0,0.0,0.0,Consumer credit,-2493,NaN
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,NaN,0.0,Consumer credit,-967,NaN
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,0.0,0.0,0.0,Consumer credit,-1508,NaN


In [7]:
df_bureau_bal

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C
...,...,...,...
27299920,5041336,-47,X
27299921,5041336,-48,X
27299922,5041336,-49,X
27299923,5041336,-50,X


In [61]:
#pd.DataFrame(df_bureau_bal[(df_bureau_bal['STATUS']!='C')&(df_bureau_bal['STATUS']!='X')].groupby(by='SK_ID_BUREAU')['STATUS'].value_counts()).rename(columns={'STATUS':'Status_count'}).reset_index()
#unstack:       pd.DataFrame(np.random.rand(5),index=[['a','s','d','f','g'],['one','two','three','four','five']]).unstack()
df_bureau_bal_f = \
    pd.DataFrame(df_bureau_bal[(df_bureau_bal['STATUS']!='C')&(df_bureau_bal['STATUS']!='X')].groupby(by='SK_ID_BUREAU')['STATUS'].value_counts()).unstack(fill_value=0).add_prefix('')
df_bureau_bal_f.reset_index(inplace=True)
df_bureau_bal_f.columns=['SK_ID_BUREAU','Status_0_count','Status_1_count','Status_2_count','Status_3_count','Status_4_count','Status_5_count']
df_bureau_bal_f.fillna(0)

,SK_ID_BUREAU,Status_0_count,Status_1_count,Status_2_count,Status_3_count,Status_4_count,Status_5_count
0,5001710,5,0,0,0,0,0
1,5001711,3,0,0,0,0,0
2,5001712,10,0,0,0,0,0
3,5001716,27,0,0,0,0,0
4,5001717,17,0,0,0,0,0
...,...,...,...,...,...,...,...
687022,6842884,9,0,0,0,0,0
687023,6842885,12,0,0,0,0,12
687024,6842886,8,0,0,0,0,0
687025,6842887,6,0,0,0,0,0


In [12]:
df_bureau_bal[   (df_bureau_bal['STATUS']!='C') & (df_bureau_bal['STATUS']!='0')    &    (df_bureau_bal['STATUS']!='X') ]   \
    ['STATUS'].value_counts()
#For df_bureau_bal , it is plan to get the count of overdue months for each ID_BUREAU for dataframe merging

1    242347
5     62406
2     23419
3      8924
4      5847
Name: STATUS, dtype: int64

In [68]:
print(df_bureau.shape)
print(df_bureau_bal_f.shape)

df_bureau_comb = pd.merge(df_bureau, df_bureau_bal_f, how='left', on=['SK_ID_BUREAU'])
df_bureau_comb[['Status_0_count','Status_1_count','Status_2_count','Status_3_count','Status_4_count','Status_5_count']]=(
    df_bureau_comb[['Status_0_count','Status_1_count','Status_2_count','Status_3_count','Status_4_count','Status_5_count']].fillna(value=0))

df_bureau_comb

(1716428, 17)
(687027, 7)


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,...,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,Status_0_count,Status_1_count,Status_2_count,Status_3_count,Status_4_count,Status_5_count
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,...,0.0,Consumer credit,-131,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,...,0.0,Credit card,-20,NaN,0.0,0.0,0.0,0.0,0.0,0.0
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,...,0.0,Consumer credit,-16,NaN,0.0,0.0,0.0,0.0,0.0,0.0
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,...,0.0,Credit card,-16,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,...,0.0,Consumer credit,-21,NaN,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,...,0.0,Microloan,-19,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,...,0.0,Consumer credit,-2493,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,...,0.0,Consumer credit,-967,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,...,0.0,Consumer credit,-1508,NaN,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
df_train.shape
df_bureau_comb.shape

df = pd.merge(df_train, df_bureau_comb, how='inner', on=['SK_ID_CURR'])

df

(307511, 122)

(1716428, 23)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY_x,...,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY_y,Status_0_count,Status_1_count,Status_2_count,Status_3_count,Status_4_count,Status_5_count
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0.0,Credit card,-1038,0.0,18.0,1.0,0.0,0.0,0.0,0.0
1,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0.0,Credit card,-47,NaN,3.0,0.0,0.0,0.0,0.0,0.0
2,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0.0,Consumer credit,-1185,0.0,5.0,6.0,0.0,0.0,0.0,0.0
3,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0.0,Consumer credit,-906,0.0,5.0,6.0,0.0,0.0,0.0,0.0
4,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0.0,Consumer credit,-34,0.0,2.0,2.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465320,456255,0,Cash loans,F,N,N,0,157500.0,675000.0,49117.5,...,0.0,Consumer credit,-25,0.0,7.0,0.0,0.0,0.0,0.0,0.0
1465321,456255,0,Cash loans,F,N,N,0,157500.0,675000.0,49117.5,...,0.0,Consumer credit,-55,3244.5,7.0,0.0,0.0,0.0,0.0,0.0
1465322,456255,0,Cash loans,F,N,N,0,157500.0,675000.0,49117.5,...,0.0,Consumer credit,-781,0.0,7.0,0.0,0.0,0.0,0.0,0.0
1465323,456255,0,Cash loans,F,N,N,0,157500.0,675000.0,49117.5,...,0.0,Credit card,-779,3244.5,7.0,0.0,0.0,0.0,0.0,0.0


In [34]:
df_CC

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3840307,1036507,328243,-9,0.000,45000,NaN,0.0,NaN,NaN,0.000,...,0.000,0.000,NaN,0,NaN,NaN,0.0,Active,0,0
3840308,1714892,347207,-9,0.000,45000,0.0,0.0,0.0,0.0,0.000,...,0.000,0.000,0.0,0,0.0,0.0,23.0,Active,0,0
3840309,1302323,215757,-9,275784.975,585000,270000.0,270000.0,0.0,0.0,2250.000,...,273093.975,273093.975,2.0,2,0.0,0.0,18.0,Active,0,0
3840310,1624872,430337,-10,0.000,450000,NaN,0.0,NaN,NaN,0.000,...,0.000,0.000,NaN,0,NaN,NaN,0.0,Active,0,0


In [76]:
df_bureau_comb.SK_ID_CURR.value_counts()

120860    116
169704     94
318065     78
251643     61
425396     60
         ... 
206292      1
216537      1
106359      1
100212      1
250544      1
Name: SK_ID_CURR, Length: 305811, dtype: int64

In [35]:
df_POS

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0
...,...,...,...,...,...,...,...,...
10001353,2448283,226558,-20,6.0,0.0,Active,843,0
10001354,1717234,141565,-19,12.0,0.0,Active,602,0
10001355,1283126,315695,-21,10.0,0.0,Active,609,0
10001356,1082516,450255,-22,12.0,0.0,Active,614,0


In [36]:
df_pre

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670209,2300464,352015,Consumer loans,14704.290,267295.5,311400.0,0.0,267295.5,WEDNESDAY,12,...,Furniture,30.0,low_normal,POS industry with interest,365243.0,-508.0,362.0,-358.0,-351.0,0.0
1670210,2357031,334635,Consumer loans,6622.020,87750.0,64291.5,29250.0,87750.0,TUESDAY,15,...,Furniture,12.0,middle,POS industry with interest,365243.0,-1604.0,-1274.0,-1304.0,-1297.0,0.0
1670211,2659632,249544,Consumer loans,11520.855,105237.0,102523.5,10525.5,105237.0,MONDAY,12,...,Consumer electronics,10.0,low_normal,POS household with interest,365243.0,-1457.0,-1187.0,-1187.0,-1181.0,0.0
1670212,2785582,400317,Cash loans,18821.520,180000.0,191880.0,NaN,180000.0,WEDNESDAY,9,...,XNA,12.0,low_normal,Cash X-Sell: low,365243.0,-1155.0,-825.0,-825.0,-817.0,1.0
